## Part 1:

Data Preparation

In [3]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

#I want to look at the games sold in 2019 and then match it with the Steam reviews
#to see if the top selling games are in fact positively received by gamers.
#I will also do sentiment analysis on the reviews texts to see how positively gamers praised these games
#and what do gamers say about the worst games.

reviews_2019= pd.read_csv('steam_reviews.csv')
reviews_2019['title']= reviews_2019['title'].str.lower().replace(u'(\N{COPYRIGHT SIGN}|\N{TRADE MARK SIGN}|\N{REGISTERED SIGN})','', regex=True)
reviews_2019['title']= reviews_2019['title'].replace("[^A-Za-z\s]*","", regex=True)
reviews_2019.head()


,date_posted,funny,helpful,hour_played,is_early_access_review,recommendation,review,title
0,2019-02-10,2,4,578,False,Recommended,&gt Played as German Reich&gt Declare war on B...,expansion hearts of iron iv man the guns
1,2019-02-10,0,0,184,False,Recommended,yes.,expansion hearts of iron iv man the guns
2,2019-02-07,0,0,892,False,Recommended,Very good game although a bit overpriced in my...,expansion hearts of iron iv man the guns
3,2018-06-14,126,1086,676,False,Recommended,Out of all the reviews I wrote This one is pro...,dead by daylight
4,2017-06-20,85,2139,612,False,Recommended,Disclaimer I survivor main. I play games for f...,dead by daylight


## Part 2:
In this part, I will aggregate all the recommended and not recommended suggestions for each game to see which games
has the most amount of recommendations from reviewers from Steam.

In [5]:
recommended= reviews_2019[reviews_2019['recommendation'] == 'Recommended'].groupby('title').size().reset_index().rename(columns={0:'Recommend'})
total_recommendations= reviews_2019.groupby('title')['recommendation'].count().reset_index().rename(columns={'recommendation':'total recommendations'})
not_recommended= reviews_2019[reviews_2019['recommendation'] == 'Not Recommended'].groupby('title').size().reset_index().rename(columns={0:'not recommended'})
recommended= recommended.merge(total_recommendations, how='left', on='title')
final_recommendation= recommended.merge(not_recommended, how='left', on='title')
final_recommendation= final_recommendation[['title','Recommend','not recommended','total recommendations']]
final_recommendation['Favorable %']= (final_recommendation['Recommend']/final_recommendation['total recommendations']*100).round()
final_recommendation['Unfavorable %']= (final_recommendation['not recommended']/final_recommendation['total recommendations']*100).round().fillna('Does not have a negative review')

# Since a lot of the games don't have any negative reviews or don't have enough reviews to make
# the game valid to analyze in the first place, I only look at the top 10 games with the most
# amount of reviews from players to avoid overfitting.
# I checked on Steam and found that Resident Evil 7: Biohazard and Astroneer received an overwhelming amount
# of positive reviews from players, so these games may be outliers in the dataset.

most_reviews= final_recommendation.sort_values(by='total recommendations', ascending=False).iloc[:10].reset_index().drop(columns='index')
most_reviews.head()

,title,Recommend,not recommended,total recommendations,Favorable %,Unfavorable %
0,playerunknowns battlegrounds,87081,58604.0,145685,60.0,40
1,grand theft auto v,58989,40967.0,99956,59.0,41
2,rust,55785,15303.0,71088,78.0,22
3,rocket league,61743,6164.0,67907,91.0,9
4,dead by daylight,17116,5105.0,22221,77.0,23


In [6]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer= SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/binhphung1003/nltk_data...


In [7]:
reviews_2019['review']= reviews_2019['review'].replace('[^A-Za-z0-9\s]*','', regex=True).apply(lambda x: str(x))
reviews_2019['sentiment score']= reviews_2019['review'].apply(lambda x: analyzer.polarity_scores(x))
reviews_2019['compound score']= reviews_2019['sentiment score'].apply(lambda dict: dict['compound'])
reviews_2019= reviews_2019.drop(columns=['date_posted', 'funny','helpful','hour_played','is_early_access_review'])
steam_ranking= reviews_2019.groupby(['title','compound score']).count().reset_index().drop(columns=['recommendation', 'review','sentiment score'])
steam_ranking= steam_ranking.groupby('title')['compound score'].mean().reset_index()
most_reviews= most_reviews.merge(steam_ranking, how='left', on='title')

In [9]:
most_reviews

,title,Recommend,not recommended,total recommendations,Favorable %,Unfavorable %,compound score
0,playerunknowns battlegrounds,87081,58604.0,145685,60.0,40,0.017397
1,grand theft auto v,58989,40967.0,99956,59.0,41,0.051204
2,rust,55785,15303.0,71088,78.0,22,0.075645
3,rocket league,61743,6164.0,67907,91.0,9,0.276007
4,dead by daylight,17116,5105.0,22221,77.0,23,0.122963
5,monster hunter world,14271,4141.0,18412,78.0,22,0.246375
6,astroneer,2573,88.0,2661,97.0,3,0.669136
7,the elder scrolls v skyrim special edition,983,490.0,1473,67.0,33,0.553139
8,resident evil biohazard re,1377,8.0,1385,99.0,1,0.474983
9,sid meiers civilization vi,351,171.0,522,67.0,33,0.373014


## Part 3:
Looking at GTA 5, PUBG 5, and RUST data, I see that the compound score is very low when the game is "Favorable", according
to the number of positive comments. Therefore, I will investigate further and see if the negative 
reviews for GTA 5, PUBG, and RUST were "extreme" (i.e. contains curse words or words that intensify the negative effects)

In [8]:
gta_5= reviews_2019[reviews_2019['title']== 'grand theft auto v'].reset_index().drop(columns=['sentiment score','compound score','title', 'index'])

In [23]:
test= gta_5['review']
test= test.apply(lambda x: nltk.word_tokenize(x))


In [24]:
test.head(10)

0     [I, love, the, part, in, Online, where, you, g...
1     [best, Rockstar, optimization, so, farI, could...
2     [After, having, bought, this, game, on, PS3, P...
3       [Pros, SP, ModdingCons, Take, Two, Interactive]
4     [Everything, in, the, game, now, either, flies...
5     [The, single, player, story, was, fun, GTA, on...
6     [Single, player, is, fun, but, online, has, be...
7     [Hey, forget, about, all, those, cool, mods, b...
8     [Great, Campaign, Awful, Customer, Service, an...
9                                           [siki, yok]
10    [gg, rockstar, you, made, a, good, game, into,...
11                                          [Its, good]
12                              [No, Mods, No, Freedom]
13    [Still, negative, because, this, game, is, jus...
14    [Shame, on, the, Developers, for, making, modd...
15    [Waited, a, year, and, a, half, for, the, PC, ...
16    [Take, Two, are, heartless, cash, grabbing, pi...
17      [Making, mods, illegal, What, a, idiotic

In [30]:
fdist= test.apply(lambda x: nltk.probability.FreqDist(x))